NB This talk is less about creating Agentic Apps as it is to demystify and simplify what AI Agents are. In this regard, this example may seem involved on seeing it the fist time. Look for where we have Day To Day Python and where we have AI, so that you may see that AI Agents are 'just' Python with LLM calls.


This example shows planning using ReAct - Reason and Act.

Essentially, we pass the output of each step to the next request to the LLM, like we did in reflection but adding in tool calling.

PLANNING is equivalent to REFLECTION and TOOL CALLING

We continue to do this until we get to the end as specified in the prompt, which we will see later in the code.

By giving examples, we are using Multi-Shot Prompting. If we gave just one example then this is called Single Shot Prompting, and no examples is Zero Shot Prompting.

We will go through it iteratively in this notebook but the next file `20_planning_agent_w_loop.py` has this as a Class containing a loop.


NB This example may need to be run through a number of times to get it to sink in so just focus on the high level view.


We will ask the agent to get the total price including VAT for a product.

It will have two tools:

1. get_product_price(product)
2. calculate_total(amount)

We will ask it to get the total price for say a laptop. It will need to REASON how to do this, take ACTION, get and OBSERVATION and repeat this loop until it gets an ANSWER.

This script has been detailed in the SYSTEM PROMPT.

*Let's not get focused on the code implementation but rather see the overall flow pattern. We can digest it more fully offline*

In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv

In [2]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if openai_api_key:
    print(f"OpenAI API Key exists and begins:{openai_api_key[:10]}")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins:sk-proj-y-


In [3]:
client = OpenAI()
MODEL = "gpt-4o-mini"

In [4]:
# We can make a class to show the ReAct (Reason - Act) pattern
class PlanningAgent:

    def __init__(self, client, model=MODEL, system: str = "") -> None:
        self.client = client

        self.system = system

        self.messages: list = []

        if self.system:

            self.messages.append({"role": "system", "content": system})

    def __call__(self, message=""):
        if message:

            self.messages.append({"role": "user", "content": message})

            result = self.execute()

            self.messages.append({"role": "assistant", "content": result})

            return result
        else:

            print("NO MESSAGE")

    def execute(self):

        completion = client.chat.completions.create(model=MODEL, messages=self.messages)

        return completion.choices[0].message.content

In [5]:
system_prompt = """
You run in a loop of THOUGHT, ACTION, PAUSE, OBSERVATION.

When you have a THOUGHT you return the THOUGHT and ACTION

At the end of the loop you output an ANSWER

Use THOUGHT to describe your THOUGHTs about the question you have been asked.

Use ACTION to run one of the ACTIONs available to you - then return PAUSE.

OBSERVATION will be the result of running those ACTIONs.

Your available ACTIONs are:

## calculate_total

if amount = 200

then calculate_total(amount)

This returns amount * 1.2

Runs the calculate_total function and returns a JSON FORMAT output as follows:

{"result": 240, "fn": "calculate_total", "next": "PAUSE"}

## get_product_price

This uses the get_product_price function and passes in the value of the product

e.g. get_product_price('bike')

This uses the get_product_price with a product = 'bike', finds the price of the bike and then returns a JSON FORMAT output as follows:

{"result": 200, "fn": "get_product_price", "next": "PAUSE"}

Example session:

Question: What is the cost of a bike??

THOUGHT: I need to find the cost of a bike|ACTION|get_product_price|bike

You will be called again with this:

OBSERVATION|200

THOUGHT: I need to calculate the total including the VAT|ACTION|calculate_total|200

You will be called again with this: 

OBSERVATION|240

If you have the answer, output it as the ANSWER.

ANSWER|The price of the bike including VAT is 240

Now it's your turn:

""".strip()


def calculate_total(amount) -> float:

    return int(amount * 1.2)


def get_product_price(product):

    if product == "bike":

        return 100

    if product == "tv":

        return 200

    if product == "laptop":

        return 300

    return None

In [6]:
agent = PlanningAgent(client=client, system=system_prompt)

In [7]:
result = agent("What is cost of a laptop including the VAT?")
# result = agent("What is cost of a bike including the VAT?")
# result = agent("What is cost of a tv including the VAT?")

print(result)

THOUGHT: I need to find the cost of a laptop|ACTION|get_product_price|laptop


We pass the result to the next call, having extracted the function name and arguments from the output.


In [8]:
next = result.split("|")
# print(next)
next_function = next[2]
next_arg = next[3]
print(f"FUNCTION - {next_function} |  ARGUMENT - {next_arg}")

FUNCTION - get_product_price |  ARGUMENT - laptop


In [9]:
if next_function == "get_product_price":
    result = get_product_price(next_arg)
    print(result)

300


In [10]:
next_prompt = f"Observation|{result}"
next_prompt

'Observation|300'

In [11]:
result = agent(next_prompt)
print(result)

THOUGHT: I need to calculate the total including the VAT|ACTION|calculate_total|300


In [12]:
next = result.split("|")
print(next)
next_function = next[2]
next_arg = int(next[3])
print(f"FUNCTION - {next_function} |  ARGUMENT - {next_arg}")

['THOUGHT: I need to calculate the total including the VAT', 'ACTION', 'calculate_total', '300']
FUNCTION - calculate_total |  ARGUMENT - 300


In [13]:
if next_function == "calculate_total":
    result = calculate_total(next_arg)
    print(result)

360


In [14]:
next_prompt = f"OBSERVATION|{result}"
next_prompt

'OBSERVATION|360'

In [15]:
result = agent(next_prompt)
print(result)

ANSWER|The price of the laptop including VAT is 360
